In [1]:
## 利用 LSTM 做一个 Seq2Seq 的预测，不考虑准确性

In [2]:
# GPU -- 准备来做一个GPU版本的LSTM做股票预测
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random


# 设置 GPU 优先
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 加载数据
dataset = pd.read_csv("000600.csv", index_col=0)
dataset = dataset.drop(['date','prediction'], axis=1)
# print(dataset.columns)
# print(dataset.tail())
dataset['updown'] = dataset['updown']
print(dataset.shape)
# print(dataset.tail())

# 方法2，将数据按照batch_size的窗口进行滑动，每个窗口数据做一组

# # 数据转成sequence的格式，这里定义每个seq的长度
seq_len = 45
batch_size = 1                                                    # 注意：batch_size是要能够整除seq_count的

# 把数据切换成 batch_size 的一个个batch
rolling_data = pd.DataFrame()
for i in dataset.rolling(seq_len):
    if i.shape[0] == seq_len:
        rolling_data = rolling_data.append(i)

rolling_data = rolling_data.values.reshape(-1, seq_len, 118)                 # 数据一共是 seq_count x seq_len x in_dim

print("rolling_data: {}".format(rolling_data.shape))
print("seq count: {}".format(rolling_data.shape[0]))                 # 所以一共有 seq_count 列数据，每一行的数据是118维 （包括y）
print("seq length: {}".format(seq_len))


total_batch_count = int(rolling_data.shape[0]/batch_size)                          # 把数据规划成 batch_count 个 batch

print("total batch count: {}".format(total_batch_count))
print("batch size: {}".format(batch_size))

rolling_data = rolling_data.reshape(total_batch_count, batch_size, seq_len, 118)  # 把数据转成 total_batch_count x batch_size x seq_len x in_dim 格式
rolling_data = torch.tensor(rolling_data)
print("rolling_data: {}".format(rolling_data.shape))


train_batch_count = total_batch_count - 1
test_batch_count = total_batch_count - train_batch_count

train = rolling_data[:train_batch_count, :, :, :]
test  = rolling_data[train_batch_count:, :, :, :]

train_x, train_y = train[:,:,:,1:], train[:,:,:,0:1]
test_x,  test_y  = test[:,:,:, 1:],  test[:,:,:,0:1]

train_x = train_x.to(device)
train_y = train_y.to(device)
test_x = test_x.to(device)
test_y = test_y.to(device)

print("train_x: {}".format(train_x.shape))
print("train_y: {}".format(train_y.shape))
print("test_x:  {}".format(test_x.shape))
print("test_y:  {}".format(test_y.shape))
print("train_batch_count: {}".format(train_batch_count))
print("test_batch_count:  {}".format(test_batch_count))

(150, 118)
rolling_data: (106, 45, 118)
seq count: 106
seq length: 45
total batch count: 106
batch size: 1
rolling_data: torch.Size([106, 1, 45, 118])
train_x: torch.Size([105, 1, 45, 117])
train_y: torch.Size([105, 1, 45, 1])
test_x:  torch.Size([1, 1, 45, 117])
test_y:  torch.Size([1, 1, 45, 1])
train_batch_count: 105
test_batch_count:  1


In [3]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size = self.hidden_dim, num_layers=self.num_layers, batch_first=True, dropout=dropout)
        # print("Encoder self.input_dim  : {}".format(self.input_dim))
        # print("Encoder self.hidden_dim  : {}".format(self.hidden_dim))
    
    def forward(self, x):
        outputs, (h_n, c_n) = self.lstm(x)
        # print("Encoder outputs :{}".format(outputs.shape))
        # print("Encoder h_n     :{}".format(h_n.shape))
        # print("Encoder c_n     :{}".format(c_n.shape))
        return outputs, h_n, c_n

    
class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        
        self.fc_in = nn.Linear(input_dim, hidden_dim)
        
        self.lstm = nn.LSTM(input_size=self.hidden_dim, hidden_size=self.hidden_dim, num_layers=self.num_layers, batch_first=True, dropout=dropout)
        
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, input, hidden, cell):
        # input : input batch data, size(input): [batch_size, feature_size]
        # notice input only has two dimensions since the input is batchs
        # of last coordinate of observed trajectory so the sequence length has been removed.
        
        # add sequence dimension to input, to allow use of nn.LSTM
        # print("Decoder forward() input size : {}".format(input.shape))
        # print("Decoder forward() hidden size: {}".format(hidden.shape))
        # print("Decoder forward() cell size  : {}".format(cell.shape))
        
        input = self.fc_in(input)

        lstm_output, (hidden, cell) = self.lstm(input, (hidden, cell))
        
        # print("Decoder forward() lstm_output: {}".format(lstm_output.shape))
        
        prediction = self.fc_out(lstm_output)         # prediction is [batch_size, output_dim]
        
        return prediction, hidden, cell
    
 

In [4]:
# Tain the model

ENC_INPUT_DIM   = 117
ENC_HIDDEN_DIM  = 768
DEC_INPUT_DIM   = 1
DEC_HIDDEN_DIM  = 768
DEC_OUPUT_DIM   = 1
NUM_LAYERS      = 2
ENC_DROPOUT     = 0.5
DEC_DROPOUT     = 0.5


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        
#         self.decoder_fc_init= nn.Linear(encoder.input_dim, decoder.input_dim)
        
#         self.decoder_fc_input= nn.Linear(decoder.output_dim, decoder.input_dim)
        
        assert (encoder.hidden_dim == decoder.hidden_dim), "hidden dimension in encoder and decoder must be equal"       
        assert (encoder.num_layers == decoder.num_layers), "hidden layer numbers in encoder and decoder must be equal"
        
            
    def forward(self, x, y):
        # x is the input to the encoder.
        # y is the output from the decoder
        # x = [batch size, encoder_in_sequence_len, encoder_in_dim]               encoder_in_sequence_len=45, encoder_in_dim=117
        # y = [batch size, decoder_out_sequence_len, decoder_out_dim]             decoder_out_sequence_len=45, decoder_out_dim=1
        
        # print("Seq2Seq forwar() x shape : {}".format(x.shape))
        # print("Seq2Seq forwar() y shape : {}".format(y.shape))
                
        batch_size = x.shape[0]
        encoder_in_seq_len = x.shape[1]
        encoder_in_dim = x.shape[2]
        
        decoder_in_dim = y.shape[2]
        decoder_out_seq_len = y.shape[1]
        
        # tensor to store decoder outputs of each time step
        outputs = torch.zeros(decoder_out_seq_len).double().to(device)
        # print("Seq2Seq forward() outputs shape: {}".format(outputs.shape))
        
        encoder_output, hidden, cell = self.encoder(x)
        
        # print("Seq2Seq forward() x[-1,-1,:] shape : {}".format(x[-1,-1,:].shape))
        # first input to decoder may be last coordinates of x
        # this is last batch and last word of sequence.
        decoder_input = torch.zeros(self.decoder.input_dim).double().to(device)
        
        # decoder_input = torch.zeros(batch_size, seq_len, OUPUT_DIM).double().to(device)
        # print("Seq2Seq forward() encoder_output shape : {}".format(encoder_output.shape))
        
        # print("Seq2Seq forward() decoder_input shape: {}".format(decoder_input.shape))
        decoder_input = decoder_input.unsqueeze(0)
        decoder_input = decoder_input.unsqueeze(0)
        # print("Seq2Seq forward() decoder_input shape: {}".format(decoder_input.shape))

        
        # Becasue the input and target have different sequence length
        # Get the target prediction one by one
        for i in range(decoder_out_seq_len):
            # run the decoder for one time step
            output, hidden, cell = self.decoder(decoder_input, hidden, cell)
            # print("Seq2Seq forward() output shape: {}".format(output.shape))
            
            # place predictions in a tensor holding predictions for each time step
            outputs[i] = output
            
            # output is the same shape as input, [batch_size, feature size]
            # so we can use output directly as next input
            decoder_input = output
            # print("Seq2Seq forward() decoder_input shape: {}".format(output.shape))
            
            # 或者使用teacher_forcing来优化
            # teacher_forcing_ratio=0.5
            # teacher_force = random.random() < teacher_forcing_ratio
            # decoder_input = y[i] if teacher_forcing else output
            
        return outputs
        

In [5]:

encoder = Encoder(input_dim=ENC_INPUT_DIM, hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, dropout=ENC_DROPOUT)
decoder = Decoder(input_dim=DEC_INPUT_DIM, hidden_dim=DEC_HIDDEN_DIM, num_layers=NUM_LAYERS, output_dim=DEC_OUPUT_DIM, dropout=DEC_DROPOUT)

model = Seq2Seq(encoder, decoder).to(device)


In [6]:
# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)
# print("The model has {:,} trainable parameters".format(count_parameters(model)))

In [7]:
LR = 1e-3
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-8)

# 训练 LSTM 模型; 
i = 0

epoches = 100

decoder_start = torch.zeros(batch_size, seq_len, 768).double().to(device)

print(time.ctime())
for epoch in range(epoches):
    for step in range(train_batch_count):
        model = model.double()

        pred = model(train_x[i], train_y[i])
        
        pred = pred.unsqueeze(0).unsqueeze(2)
        
        # print("Train pred shape : {}".format(pred.shape))
        # print("Train train_y[i] shape : {}".format(train_y[i].shape))
        
        loss = loss_func(pred, train_y[i])
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=20, norm_type=2)
        optimizer.step()

        if (i+1)%train_batch_count == 0:
            i=0
        else:
            i=i+1
    if (epoch+1)%10 == 0:
        print("{} of {} epoch loss: {:.6f}".format(epoch, epoches, loss.item()))
        # print("Prediction: {}".format(pred.data))
        # print("Actual Res: {}".format(train_y[i].data))

print(time.ctime())

Thu Mar  3 15:29:00 2022
9 of 100 epoch loss: 0.000313
19 of 100 epoch loss: 0.000317
29 of 100 epoch loss: 0.000339
39 of 100 epoch loss: 0.000320
49 of 100 epoch loss: 0.000326
59 of 100 epoch loss: 0.000307
69 of 100 epoch loss: 0.000289


KeyboardInterrupt: 